In [4]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# datat = spcm.DataTransfer(card)

# Setup the channels
channels = spcm.Channels(card)
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(1. * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDSCommandList(card)
# dds = spcm.DDS(card)
dds.reset()

dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
# dds.phase_behaviour(0)

core_list = [hex(2**n) for n in range(20)]

# dds.mode = dds.WRITE_MODE.WAIT_IF_FULL

dds.write_to_card()

card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

SpcmException: 

In [10]:
clk = spcm.Clock(card)
print(clk.sample_rate())
print(clk.max_sample_rate())

1250000000
1250000000


In [21]:
1/(200.e3)

5e-06

In [3]:
def split_array(f_split = 2.4e6, f_width_single_side = 4.e6, n_per_side = 4):
    f_center = 74.8e6
    # array1 = np.linspace( f_center-f_split/2-f_width_single_side ,f_center-f_split/2,n_per_side)
    array1 = f_center - f_split/2 - np.linspace( f_width_single_side , 0. ,n_per_side)
    # array2 = np.linspace( f_center+f_split/2+f_width_single_side ,f_center+f_split/2,n_per_side)
    array2 = f_center + f_split/2 + np.linspace( 0., f_width_single_side, n_per_side)
    # array1 = np.linspace(70.5e6,72.5e6,n)
    # array2 = np.linspace(77.5e6,79.5e6,n)
    return np.concatenate([array1,array2])

def compute_tweezer_1064_phases(n_tweezers,amplitudes):
    phase_tweezer_array = np.empty([n_tweezers])
    for tweezer_idx in range(n_tweezers):
        if tweezer_idx == 0:
            phase_tweezer_array[0] = 360
        else:
            phase_ij = 0
            for j in range(1,tweezer_idx):
                phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*amplitudes[tweezer_idx]
            phase_i = (phase_ij % 2*np.pi) * 360
            phase_tweezer_array[tweezer_idx] = phase_i
    return phase_tweezer_array

# f_list = split_array(f_split=2.4e6)
f_list = [71.6e6,76.e6,80.e6]
# f_list = [73.6e6,76.e6]

n_tweezers = len(f_list)

# a_list = np.linspace(1/n_tweezers,1/n_tweezers,n_tweezers)
# a_list = [.1,.1,.1,.05,.1,.35]
a_list = [.48,.33,.18]

phases = compute_tweezer_1064_phases(len(f_list),a_list)

def normalize_alist(alist):
    sum = np.sum(alist)
    return a_list/sum
a_list = normalize_alist(a_list)

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
        dds[tweezer_idx].phase(phases[tweezer_idx])
    else:
        dds[tweezer_idx].amp(0.)

dds.exec_at_trg()
dds.write_to_card()
dds.write()
trigger.force()

In [4]:
# compute tweezer movement params
dpf = 5.7e-12 # m per Hz

# which tweezer (give idx of desired tone in f_list)
which_tweezer = 0

# amplitude
amp = 1.e6 # Hz

# mod frequency Hz
f_mod = 200.e3

# how long to modulate for
t_total = .01

dt = .5e-6

# functional form
def sin(t,amplitude,frequency):
    return amplitude*np.sin(2*np.pi*frequency*t)

dts = np.linspace(0,t_total,int(t_total/dt))

slopes = np.diff(sin(dts,amp,f_mod)) / dt

stop_array = np.array([0.])

slopes = np.concatenate([slopes,stop_array])

In [4]:
print(len(slopes))

200000


In [5]:
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(dt)
dds.exec_at_trg()
dds.write_to_card()

dds.load({spcm.SPC_DDS_CORE0_FREQ_SLOPE:slopes},exec_mode=spcm.SPCM_DDS_CMD_EXEC_AT_TRG, repeat=1)

dds.write()

dds.mode = dds.WRITE_MODE.WAIT_IF_FULL

# dds.frequency_slope(0,0.)
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()

dds.write()

In [8]:
# execute single movement

# start trigger timer, which outputs trigger events at a given rate
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
dds.trg_timer(dt)
dds.exec_at_trg()
# dds.write_to_card()

# write slopes to card
for slope in slopes:
    dds.frequency_slope(which_tweezer,slope)
    dds.exec_at_trg()
# dds.write_to_card()
dds.frequency_slope(which_tweezer,0.)
# reset trigger mode to external at the end
dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)
dds.exec_at_trg()
dds.write_to_card()


In [10]:
trigger.force()

In [6]:
# card.close(card._handle)
card.stop()

card.close(card._handle)